# IMDB XGBoost Total Score
![ImdbIcon](../images/imdbheader.jpg)

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import linear_model
from sklearn.model_selection import cross_val_score, train_test_split, GridSearchCV

# Importing so that I can use LinearRegression and OLS Models.
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn import metrics

In [2]:
final_df = pd.read_csv('../data/totalscore_df.csv')

In [3]:
final_df.columns

Index(['movie_title', 'year', 'actors', 'plot', 'duration', 'Action',
       'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary',
       'Drama', 'Family', 'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music',
       'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 'Sci-Fi',
       'Sport', 'Thriller', 'War', 'Western', 'avg_vote', 'votes',
       'weighted_average_vote', 'total_votes', 'mean_vote', 'median_vote',
       'votes_1', 'votes_2', 'votes_3', 'votes_4', 'votes_5', 'votes_6',
       'votes_7', 'votes_8', 'votes_9', 'votes_10', 'us_voters_rating',
       'us_voters_votes', 'plot_sentiment', 'director_score', 'actor_score',
       'actress_score', 'total_score'],
      dtype='object')

### XGBoost

In [4]:
from xgboost import XGBRegressor, XGBClassifier

In [5]:
xgb = XGBRegressor()

In [41]:
features = ['duration', 'Action', 'Adventure', 'Animation', 'Biography', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Family', 
            'Fantasy', 'Film-Noir', 'History', 'Horror', 'Music', 'Musical', 'Mystery', 'News', 'Reality-TV', 'Romance', 
            'Sci-Fi', 'Sport', 'Thriller', 'War', 'Western',
             'plot_sentiment', 'votes_1', 
            'director_score', 'actor_score', 'actress_score']

In [42]:
X = final_df[features]
y = final_df['total_score']

In [43]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state = 42)

In [44]:
xgb.fit(X_train, y_train)

XGBRegressor(base_score=0.5, booster='gbtree', colsample_bylevel=1,
             colsample_bynode=1, colsample_bytree=1, gamma=0, gpu_id=-1,
             importance_type='gain', interaction_constraints='',
             learning_rate=0.300000012, max_delta_step=0, max_depth=6,
             min_child_weight=1, missing=nan, monotone_constraints='()',
             n_estimators=100, n_jobs=24, num_parallel_tree=1, random_state=0,
             reg_alpha=0, reg_lambda=1, scale_pos_weight=1, subsample=1,
             tree_method='exact', validate_parameters=1, verbosity=None)

In [45]:
print("Training Score:", xgb.score(X_train, y_train))
print(" ")
print("Testing Score:", xgb.score(X_test, y_test))

Training Score: 0.9209672949820484
 
Testing Score: 0.7788125035476363


In [46]:
predictions = xgb.predict(X_train)

residuals = y_train - predictions

rss = (residuals ** 2).sum()

mse = metrics.mean_squared_error(y_train, predictions)

rmse = np.sqrt(metrics.mean_squared_error(y_train, predictions))

In [47]:
null_predictions = np.zeros_like(y_test)

null_pred = null_predictions + y_test.mean()

baseline = metrics.mean_squared_error(y_test, null_pred, squared = False)

In [48]:
print("Baseline RMSE:", baseline)
print(" ")
print("RMSE:", rmse)
print("________________________________")
print(" ")
print("Difference:", baseline - rmse)

Baseline RMSE: 1.3266860099257332
 
RMSE: 0.37086307860766515
________________________________
 
Difference: 0.955822931318068


In [49]:
print("RMSE:", rmse)
print(" ")
print("Training Score:", xgb.score(X_train, y_train))
print(" ")
print("Testing Score:", xgb.score(X_test, y_test))

RMSE: 0.37086307860766515
 
Training Score: 0.9209672949820484
 
Testing Score: 0.7788125035476363


In [50]:
xgb = XGBRegressor()

In [51]:
model_params = {}
count = 0

In [53]:
xgb_params = {
    'base_score' : [.05, .10, .15],
    'booster' : ['gbtree', 'gblinear', 'dart'],
    'learning_rate': [.10, .15, .20],
    'max_depth' : [3, 4, 5],
    'n_estimators': [150, 200, 250],
}

gs = GridSearchCV(xgb, param_grid = xgb_params, cv = 5, n_jobs = 12)

gs.fit(X_train, y_train)

count += 1

gs.best_params_['best_score'] = gs.best_score_

gs.best_params_['training_score'] = gs.score(X_train, y_train)

gs.best_params_['testing_score'] = gs.score(X_test, y_test)

model_params[f'model_{count}'] = gs.best_params_

model_df = pd.DataFrame.from_dict(model_params, orient = 'index')

model_df

,base_score,booster,learning_rate,max_depth,n_estimators,best_score,training_score,testing_score
model_1,0.1,dart,0.15,3,200,0.801190,0.839779,0.788780
model_2,0.1,gbtree,0.10,5,150,0.803848,0.863247,0.791022
